<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/wordmark_black65ee464.png" width="700">

# Week 2 : Final Challenge

**Welcome to the final challenge!**  

In the previous notebook we've seen how we can use the VQC class in Aqua to classify the digits `0` and `1`. However, classifying `0` and `1` is relatively simple as digits `0` and `1` are easily distinguishable. `4` and `9` however, are notoriously similar, with a _loop_ on the top and a _line_ on the bottom. This can be corroborated looking at our 2-D t-SNE plot from the previous notebook (Fig.2), we see that `0` and `1` are clustered relatively far from each other making them easily distinguishable, however `4` and `9` are overlapping. In this challenge we are providing you with a dataset with digits reduced to **dimension 3**. For example, in Fig.1 we can see the dimension reduction of the 784 dimension vector for digit `4` into a dimension 3 feature vector. 

**Fig.1 : Features of the digit `4` after reducing dimension to 3:** 
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/four2a7701f.png" width="700">

**Fig.2 : MNIST dataset after dimension reduction to 2 as given in the previous notebook:**
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/mnist_plot53adb39.png" width="400">

## Challenge Question   
Use the VQC method from Aqua to classify the digits `4` and `9` as given in the dataset **challenge_dataset_4_9_dim3_training.csv** provided to you. 

## Rules and Guidelines

* Your `QuantumCircuit` can have a **maximum of 6 qubits**.
* **Cost of the circuit should be less than 2000**.  
* You should not change names of the functions `feature_map()` , `variational_circuit()`  and `return_optimal_params()`.
* All the functions must return the value types as mentioned. 
* All circuits must be Qiskit generated.
* Best of all submissions is considered for grading.

## Judging criteria 

* Primary judgement is based on the **accuracy of the model**, higher the better. **Accuracies which differ by less than 0.005 will be considered to be equal**. ex: Accuracies 0.7783 and 0.7741 will be considered to be equal.
* If the accuracies are tied, the tie will be broken using **cost of the circuit** as the metric, lower the better. 
* In the case that both accuracy of the model and cost of the circuit are equal, **time of submission** is taken into account, Earlier the better. 

_**Important Note:**_ The **leaderboard shown during the progress of the competition** will only display accuracy of the model and is **not the final leaderboard**. Breaking ties between accuracy of the model by considering lower **cost of circuit** will only be done after the competition ends. **The final leaderboard will be announced post the event** which will take into consideration cost of the circuit and time of submission. 

## Certificate Eligibility

Everyone who scores an **accuracy greater than 0.70 (i.e, 70%) will be eligible for a certificate**. 


An explanation on how to calculate the accuracy of the model and the cost of the circuit is given in the end inside the `grade()` function. Before you submit, make sure the grading function is running on your device. To save time you can also use the grading function provided to calculate the accuracy and circuit cost without having to submit your solution onto HackerEarth. Remember, your final score will be determined using the same grading methods as given in this notebook, but will be evaluated on unseen datapoints.

In [1]:
# installing a few dependencies
# !pip install --upgrade seaborn==0.10.1
# !pip install --upgrade scikit-learn==0.23.1
# !pip install --upgrade matplotlib==3.2.0
# !pip install --upgrade pandas==1.0.4
# !pip install --upgrade qiskit==0.19.6 
# !pip install --upgrade plotly==4.9.0

# the output will be cleared after installation
from IPython.display import clear_output
clear_output()

In [2]:
# we have imported a few libraries we thing might be useful 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer

from qiskit import *
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
%matplotlib inline
import matplotlib.pyplot as plt

import time
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, RealAmplitudes, EfficientSU2
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import COBYLA


# The the write_and_run() magic function creates a file with the content inside the cell that it is run. 
# You have used this in previous exercises for creating your submission files. 
# It will be used for the same purpose here.

from IPython.core.magic import register_cell_magic
@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

# Solution

## Data loading 

This notebook has helper functions and code snippets to save your time and help you concentrate on what's important: Increasing the accuracy of your model. Running the cell below will import the challenge dataset and will be available to you as `data`. Before running the cell below store the dataset in this file structure (or change the `data_path` accordingly):  

- `challenge_notebook.ipynb`
- `dataset`
    - `challenge_dataset_4_9.csv`


In [3]:
#data_path='./dataset/'
data_path='../'
data = np.loadtxt(data_path + "challenge_dataset_4_9_dim3_training.csv", delimiter=",")

# extracting the first column which contains the labels
data_labels = data[:, :1].reshape(data.shape[0],)
# extracting all the columns but the first which are our features
data_features = data[:, 1:]

## Visualizing the dataset

Before we dive into solving the question it is always beneficial to look at the dataset pictographically. This will help us understand patterns which we could leverage when designing our feature maps and variational circuits for example.

## Extracting the training dataset

The given dataset has already been reduced in dimension and normalized, so, further pre-processing isn't techincally required. You can do so if you want to, but the testing dataset will be of the same dimension and normalisation as the training dataset provided. Training a dataset of size 6,000 will take multiple hours so you'll need to extract a subset of the dataset to use as a training dataset. The accuracy of the model may vary based on the datapoints and size of the training dataset you choose. Thus, experimenting with various sizes and datapoints will be necessary. For example, Increasing the training dataset size may increase the accuracy of the model however it will increase the training time as well.

Use the space below to extract your training dataset from `data`. For your convenience `data` has been segregated into `data_labels` and `data_features`.

* `data_labels` : 6,000 $\times$ 1 column vector with each entry either `4` or `9` 
* `data_features` : 6,000 $\times$ 3 matrix with each row having the feature corresponding to the label in `data_labels`

**Note:** This process was done in the previous [VQC notebook](https://github.com/Qiskit-Challenge-India/2020/blob/master/Day%206%2C%207%2C8/VQC_notebook.ipynb) with `0` and `1` labels and can be modified and used here as well. 

In [13]:
### WRITE YOUR CODE BETWEEN THESE LINES - START

# do your classical pre-processing here

# store your training and testing datasets to be input in the VQC optimizer in the "training_input" and 
# "testing_input" variables respectively. These variables will eb accessed whiile creating a VQC instance later. 
four_datapoints = []
nine_datapoints = []

for i in range(6000):
    if data_labels[i] == 4:
        four_datapoints.append(data_features[i])

for i in range(6000):
    if data_labels[i] == 9:
        nine_datapoints.append(data_features[i])
        
four_datapoints = np.array(four_datapoints)
nine_datapoints = np.array(nine_datapoints)
np.random.shuffle(four_datapoints)
np.random.shuffle(nine_datapoints)

train_num = 100
test_num = 100

training_input = {4:four_datapoints[:train_num], 9:nine_datapoints[:train_num]}
test_input = {4:four_datapoints[-test_num:], 9:nine_datapoints[-test_num:]}

### WRITE YOUR CODE BETWEEN THESE LINES - END

## Building a Quantum Feature Map

Given below is the `feature_map()` function. It takes no input and has to return a feature map which is either a `FeatureMap` or `QuantumCircuit` object. In the previous notebook you've learnt how feature maps work and the process of using existing feature maps in Qiskit or creating your own. In the space given **inside the function** you have to create a feature map and return it.   


**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `feature_map.py`

In [5]:
%%write_and_run feature_map.py

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

def feature_map(): 

    ckt9_depth=3
    num_qubits=6
    x = ParameterVector('x', length=3)
    feature_map=QuantumCircuit(num_qubits)
    
    for i in range(ckt9_depth):
        for j in range(num_qubits):
            feature_map.h(j)
        for j in range(2):
            feature_map.crz((np.pi-x[j])*(np.pi-x[j+1]),j,j+1)
        for j in range(2):
            feature_map.crz((np.pi-x[j]*x[j])*(np.pi-x[j+1]*x[j+1]),j+3,j+4)
        for j in range(3):
            feature_map.rx(x[j],j)
        for j in range(3):
            feature_map.rx(x[j]*x[j],j+3)

    #return the feature map which is either a FeatureMap or QuantumCircuit object
    return feature_map

In [6]:
feature_map().draw()

┌───┐                                                                 »
q_0: ┤ H ├────────────────────────────────■────────────────────────────────»
     ├───┤   ┌────────────────────────────┴────────────────────────────┐   »
q_1: ┤ H ├───┤ RZ((3.14159265358979 - x[0])*(3.14159265358979 - x[1])) ├───»
     ├───┤   └─────────────────────────────────────────────────────────┘   »
q_2: ┤ H ├─────────────────────────────────────────────────────────────────»
     ├───┤                                                                 »
q_3: ┤ H ├────────────────────────────────■────────────────────────────────»
     ├───┤┌───────────────────────────────┴───────────────────────────────┐»
q_4: ┤ H ├┤ RZ((3.14159265358979 - x[0]**2)*(3.14159265358979 - x[1]**2)) ├»
     ├───┤└───────────────────────────────────────────────────────────────┘»
q_5: ┤ H ├─────────────────────────────────────────────────────────────────»
     └───┘                                                                 »
«                                ┌──────────┐                          »
«q_0: ───────────────────────────┤ RX(x[0]) ├──────────────────────────»
«                                └──────────┘                          »
«q_1: ────────────────────────────────■────────────────────────────────»
«        ┌────────────────────────────┴────────────────────────────┐   »
«q_2: ───┤ RZ((3.14159265358979 - x[1])*(3.14159265358979 - x[2])) ├───»
«        └─────────────────────┬─────────────┬─────────────────────┘   »
«q_3: ─────────────────────────┤ RX(x[0]**2) ├─────────────────────────»
«                              └─────────────┘                         »
«q_4: ────────────────────────────────■────────────────────────────────»
«     ┌───────────────────────────────┴───────────────────────────────┐»
«q_5: ┤ RZ((3.14159265358979 - x[1]**2)*(3.14159265358979 - x[2]**2)) ├»
«     └───────────────────────────────────────────────────────────────┘»
«          ┌───┐          »
«q_0: ─────┤ H ├──────────»
«       ┌──┴───┴───┐ ┌───┐»
«q_1: ──┤ RX(x[1]) ├─┤ H ├»
«       ├──────────┤ ├───┤»
«q_2: ──┤ RX(x[2]) ├─┤ H ├»
«       └──┬───┬───┘ └───┘»
«q_3: ─────┤ H ├──────────»
«     ┌────┴───┴────┐┌───┐»
«q_4: ┤ RX(x[1]**2) ├┤ H ├»
«     ├─────────────┤├───┤»
«q_5: ┤ RX(x[2]**2) ├┤ H ├»
«     └─────────────┘└───┘»
«                                                                      »
«q_0: ────────────────────────────────■────────────────────────────────»
«        ┌────────────────────────────┴────────────────────────────┐   »
«q_1: ───┤ RZ((3.14159265358979 - x[0])*(3.14159265358979 - x[1])) ├───»
«        └─────────────────────────────────────────────────────────┘   »
«q_2: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«q_3: ────────────────────────────────■────────────────────────────────»
«     ┌───────────────────────────────┴───────────────────────────────┐»
«q_4: ┤ RZ((3.14159265358979 - x[0]**2)*(3.14159265358979 - x[1]**2)) ├»
«     └───────────────────────────────────────────────────────────────┘»
«q_5: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«                                ┌──────────┐                          »
«q_0: ───────────────────────────┤ RX(x[0]) ├──────────────────────────»
«                                └──────────┘                          »
«q_1: ────────────────────────────────■────────────────────────────────»
«        ┌────────────────────────────┴────────────────────────────┐   »
«q_2: ───┤ RZ((3.14159265358979 - x[1])*(3.14159265358979 - x[2])) ├───»
«        └─────────────────────┬─────────────┬─────────────────────┘   »
«q_3: ─────────────────────────┤ RX(x[0]**2) ├─────────────────────────»
«                              └─────────────┘                         »
«q_4: ────────────────────────────────■────────────────────────────────»
«     ┌─────

## Building a Variational Circuit

Given below is the `variational_circuit()` function. It takes no input and has to return a variational circuit which is either a `VariationalForm` or `QuantumCircuit` object. In the previous notebook you've learnt how variational circuits work and the process of using existing variational circuit in Qiskit or creating your own. You have to create a variational circuit in the space given **inside the function** and return it. You can find various variational circuits in the [Qiskit Circuit Library](https://qiskit.org/documentation/apidoc/circuit_library.html) under N-local circuits.

**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `variational_circuit.py`

In [7]:
%%write_and_run variational_circuit.py

def variational_circuit():

    ckt5_depth=6
    num_qubits=3
    y = ParameterVector('y', length=num_qubits*(num_qubits+3)*ckt5_depth)
    ckt5_depth=3
    num_qubits=6
    var_circuit=QuantumCircuit(num_qubits)
    ii = 0
    
    for i in range(ckt5_depth):
        for j in range(num_qubits):
            var_circuit.rx(y[ii], j)
            ii += 1
        for j in range(num_qubits):
            var_circuit.rz(y[ii], j)
            ii += 1
        for j in range(num_qubits//2):
            for k in range(num_qubits//2):
                if j != k:
                    var_circuit.crz(y[ii], num_qubits-1-j, num_qubits-1-k)
                    ii += 1
        for j in range(num_qubits//2):
            for k in range(num_qubits//2):
                if j != k:
                    var_circuit.crz(y[ii], num_qubits-1-j-3, num_qubits-1-k-3)
                    ii += 1
        for j in range(num_qubits):
            var_circuit.rx(y[ii], j)
            ii += 1
        for j in range(num_qubits):
            var_circuit.rz(y[ii], j)
            ii += 1

    # return the variational circuit which is either a VaritionalForm or QuantumCircuit object
    return var_circuit

In [8]:
variational_circuit().draw()

┌──────────┐ ┌──────────┐             ┌───────────┐             »
q_0: ┤ RX(y[0]) ├─┤ RZ(y[6]) ├─────────────┤ RZ(y[19]) ├─────────────»
     ├──────────┤ ├──────────┤┌───────────┐└─────┬─────┘             »
q_1: ┤ RX(y[1]) ├─┤ RZ(y[7]) ├┤ RZ(y[18]) ├──────┼────────────■──────»
     ├──────────┤ ├──────────┤└─────┬─────┘      │      ┌─────┴─────┐»
q_2: ┤ RX(y[2]) ├─┤ RZ(y[8]) ├──────■────────────■──────┤ RZ(y[20]) ├»
     ├──────────┤ ├──────────┤             ┌───────────┐└───────────┘»
q_3: ┤ RX(y[3]) ├─┤ RZ(y[9]) ├─────────────┤ RZ(y[13]) ├─────────────»
     ├──────────┤┌┴──────────┤┌───────────┐└─────┬─────┘             »
q_4: ┤ RX(y[4]) ├┤ RZ(y[10]) ├┤ RZ(y[12]) ├──────┼────────────■──────»
     ├──────────┤├───────────┤└─────┬─────┘      │      ┌─────┴─────┐»
q_5: ┤ RX(y[5]) ├┤ RZ(y[11]) ├──────■────────────■──────┤ RZ(y[14]) ├»
     └──────────┘└───────────┘                          └───────────┘»
«     ┌───────────┐                          ┌───────────┐┌───────────┐»
«q_0: ┤ RZ(y[21]) ├──────■────────────■──────┤ RX(y[24]) ├┤ RZ(y[30]) ├»
«     └─────┬─────┘      │      ┌─────┴─────┐├───────────┤├───────────┤»
«q_1: ──────■────────────┼──────┤ RZ(y[23]) ├┤ RX(y[25]) ├┤ RZ(y[31]) ├»
«                  ┌─────┴─────┐├───────────┤├───────────┤├───────────┤»
«q_2: ─────────────┤ RZ(y[22]) ├┤ RX(y[26]) ├┤ RZ(y[32]) ├┤ RX(y[38]) ├»
«     ┌───────────┐└───────────┘└───────────┘├───────────┤├───────────┤»
«q_3: ┤ RZ(y[15]) ├──────■────────────■──────┤ RX(y[27]) ├┤ RZ(y[33]) ├»
«     └─────┬─────┘      │      ┌─────┴─────┐├───────────┤├───────────┤»
«q_4: ──────■────────────┼──────┤ RZ(y[17]) ├┤ RX(y[28]) ├┤ RZ(y[34]) ├»
«                  ┌─────┴─────┐├───────────┤├───────────┤├───────────┤»
«q_5: ─────────────┤ RZ(y[16]) ├┤ RX(y[29]) ├┤ RZ(y[35]) ├┤ RX(y[41]) ├»
«                  └───────────┘└───────────┘└───────────┘└───────────┘»
«     ┌───────────┐┌───────────┐             ┌───────────┐             »
«q_0: ┤ RX(y[36]) ├┤ RZ(y[42]) ├─────────────┤ RZ(y[55]) ├─────────────»
«     ├───────────┤├───────────┤┌───────────┐└─────┬─────┘             »
«q_1: ┤ RX(y[37]) ├┤ RZ(y[43]) ├┤ RZ(y[54]) ├──────┼────────────■──────»
«     ├───────────┤└───────────┘└─────┬─────┘      │      ┌─────┴─────┐»
«q_2: ┤ RZ(y[44]) ├───────────────────■────────────■──────┤ RZ(y[56]) ├»
«     ├───────────┤┌───────────┐             ┌───────────┐└───────────┘»
«q_3: ┤ RX(y[39]) ├┤ RZ(y[45]) ├─────────────┤ RZ(y[49]) ├─────────────»
«     ├───────────┤├───────────┤┌───────────┐└─────┬─────┘             »
«q_4: ┤ RX(y[40]) ├┤ RZ(y[46]) ├┤ RZ(y[48]) ├──────┼────────────■──────»
«     ├───────────┤└───────────┘└─────┬─────┘      │      ┌─────┴─────┐»
«q_5: ┤ RZ(y[47]) ├───────────────────■────────────■──────┤ RZ(y[50]) ├»
«     └───────────┘                                       └───────────┘»
«     ┌───────────┐                          ┌───────────┐┌───────────┐»
«q_0: ┤ RZ(y[57]) ├──────■────────────■──────┤ RX(y[60]) ├┤ RZ(y[66]) ├»
«     └─────┬─────┘      │      ┌─────┴─────┐├───────────┤├───────────┤»
«q_1: ──────■────────────┼──────┤ RZ(y[59]) ├┤ RX(y[61]) ├┤ RZ(y[67]) ├»
«                  ┌─────┴─────┐├───────────┤├───────────┤├───────────┤»
«q_2: ─────────────┤ RZ(y[58]) ├┤ RX(y[62]) ├┤ RZ(y[68]) ├┤ RX(y[74]) ├»
«     ┌───────────┐└───────────┘└───────────┘├───────────┤├───────────┤»
«q_3: ┤ RZ(y[51]) ├──────■────────────■──────┤ RX(y[63]) ├┤ RZ(y[69]) ├»
«     └─────┬─────┘      │      ┌─────┴─────┐├───────────┤├───────────┤»
«q_4: ──────■────────────┼──────┤ RZ(y[53]) ├┤ RX(y[64]) ├┤ RZ(y[70]) ├»
«                  ┌─────┴─────┐├───────────┤├───────────┤├───────────┤»
«q_5: ─────────────┤ RZ(y[52]) ├┤ RX(y[65]) ├┤ RZ(y[71]) ├┤ RX(y[77]) ├»
«                  └───────────┘└───────────┘└───────────┘└───────────┘»
«     ┌───────────┐┌───────────┐             ┌───────────┐             »
«q_0: ┤ RX(y[72]) ├┤ RZ(y[78]) ├─────────────┤ RZ(y[91]) ├─────────────»
«     ├───────────┤├───────────┤┌───────────┐└─────┬─────┘             »
«q_1: ┤ RX(y[73]

## Choosing a Classical Optimizer

In the `classical_optimizer()` function given below you will have to import the optimizer of your choice from [`qiskit.aqua.optimizers`](https://qiskit.org/documentation/apidoc/qiskit.aqua.components.optimizers.html) and return it. This function will not be called by the grading function `grade()` and thus the name of the function `classical_optimizer()`can be changed if needed. 

In [9]:
def classical_optimizer():
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - START
    
    # import the required clasical optimizer from qiskit.aqua.optimizers
    from qiskit.aqua.components.optimizers import SPSA
    
    # create an optimizer object
    cls_opt = COBYLA(maxiter=500, tol=0.001)
    
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - END
    return cls_opt

### Callback Function

The `VQC` class can take in a callback function to which the following parameters will be passed after every optimization cycle of the algorithm:

* `eval_count` : the evaulation counter
* `var_params` : value of parameters of the variational circuit
* `eval_val`  : current cross entropy cost 
* `index` : the batch index

In [10]:
def call_back_vqc(eval_count, var_params, eval_val, index):
    print("eval_count: {}".format(eval_count))
    print("var_params: {}".format(var_params))
    print("eval_val: {}".format(eval_val))
    print("index: {}".format(index))

## Optimization Step

This is where the whole VQC algorithm will come together. First we create an instance of the `VQC` class. 

In [11]:
# a fixed seed so that we get the same answer when the same input is given. 
seed = 1059

# setting our backend to qasm_simulator with the "statevector" method on. This particular setup is given as it was 
# found to perform better than most. Feel free to play around with different backend options.
backend = Aer.get_backend('qasm_simulator')
backend_options = {"method": "statevector"}

# creating a quantum instance using the backend and backend options taken before
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed, 
                                   backend_options=backend_options)
#quantum_instance = QuantumInstance(backend, shots=1024, backend_options=backend_options)

# creating a VQC instance which you will be used for training. Make sure you input the correct training_dataset and 
# testing_dataset as defined in your program.
vqc = VQC(optimizer=classical_optimizer(), 
          feature_map=feature_map(), 
          var_form=variational_circuit(), 
          callback=call_back_vqc, 
          training_dataset=training_input,     # training_input must be initialized with your training dataset
          test_dataset=test_input)             # testing_input must be initialized with your testing dataset

Now, let's run the VQC classification routine

In [ ]:
start = time.process_time()

result = vqc.run(quantum_instance)

print("time taken: ")
print(time.process_time() - start)

print("testing success ratio: {}".format(result['testing_accuracy']))

eval_count: 0
var_params: [ 6.85676591e-01 -9.12514107e-01  1.08825401e+00  1.73740900e+00
 -1.14979269e+00 -5.62775003e-01  2.28232149e-01  3.83038064e-01
  1.25270709e+00  5.97235423e-01  2.30209944e-01 -1.29585080e-01
 -6.36766422e-01  2.26276696e+00 -7.15756632e-01 -6.21057867e-01
 -2.04856182e+00  1.03594956e+00  1.74330162e+00 -1.13141354e+00
 -1.02552510e-02 -1.08535631e+00  1.68004572e+00 -5.76162205e-01
 -1.78430883e+00 -2.21026995e-01  1.46152685e-01  1.07478206e+00
 -4.59351845e-02 -4.78414666e-02 -1.52263642e+00  4.84891687e-01
  2.40517337e+00 -3.35079932e-02  1.09506099e+00 -4.69896107e-01
 -4.38737636e-01  1.26083289e-01  7.33813010e-01  1.51476739e+00
  3.70235102e-01  5.98917531e-01 -3.46029133e-01 -1.00987786e+00
 -5.83920218e-01  1.00375635e+00 -2.76504727e-01  3.33913200e-01
 -6.88313164e-01 -4.59709270e-01 -1.38847882e+00 -2.11447196e+00
  1.62353604e-01 -8.17103070e-01 -6.18470410e-01  1.08539315e+00
 -9.12837795e-01  3.32591955e-01 -9.37398183e-01  3.80382688e-02

eval_count: 5
var_params: [ 6.85676591e-01 -9.12514107e-01  1.08825401e+00  1.73740900e+00
 -1.49792691e-01 -5.62775003e-01  2.28232149e-01  3.83038064e-01
  1.25270709e+00  5.97235423e-01  2.30209944e-01 -1.29585080e-01
 -6.36766422e-01  2.26276696e+00 -7.15756632e-01 -6.21057867e-01
 -2.04856182e+00  1.03594956e+00  1.74330162e+00 -1.13141354e+00
 -1.02552510e-02 -1.08535631e+00  1.68004572e+00 -5.76162205e-01
 -1.78430883e+00 -2.21026995e-01  1.46152685e-01  1.07478206e+00
 -4.59351845e-02 -4.78414666e-02 -1.52263642e+00  4.84891687e-01
  2.40517337e+00 -3.35079932e-02  1.09506099e+00 -4.69896107e-01
 -4.38737636e-01  1.26083289e-01  7.33813010e-01  1.51476739e+00
  3.70235102e-01  5.98917531e-01 -3.46029133e-01 -1.00987786e+00
 -5.83920218e-01  1.00375635e+00 -2.76504727e-01  3.33913200e-01
 -6.88313164e-01 -4.59709270e-01 -1.38847882e+00 -2.11447196e+00
  1.62353604e-01 -8.17103070e-01 -6.18470410e-01  1.08539315e+00
 -9.12837795e-01  3.32591955e-01 -9.37398183e-01  3.80382688e-02

eval_count: 10
var_params: [ 6.85676591e-01 -9.12514107e-01  1.08825401e+00  1.73740900e+00
 -1.14979269e+00 -5.62775003e-01  2.28232149e-01  3.83038064e-01
  1.25270709e+00  1.59723542e+00  2.30209944e-01 -1.29585080e-01
 -6.36766422e-01  2.26276696e+00 -7.15756632e-01 -6.21057867e-01
 -2.04856182e+00  1.03594956e+00  1.74330162e+00 -1.13141354e+00
 -1.02552510e-02 -1.08535631e+00  1.68004572e+00 -5.76162205e-01
 -1.78430883e+00 -2.21026995e-01  1.46152685e-01  1.07478206e+00
 -4.59351845e-02 -4.78414666e-02 -1.52263642e+00  4.84891687e-01
  2.40517337e+00 -3.35079932e-02  1.09506099e+00 -4.69896107e-01
 -4.38737636e-01  1.26083289e-01  7.33813010e-01  1.51476739e+00
  3.70235102e-01  5.98917531e-01 -3.46029133e-01 -1.00987786e+00
 -5.83920218e-01  1.00375635e+00 -2.76504727e-01  3.33913200e-01
 -6.88313164e-01 -4.59709270e-01 -1.38847882e+00 -2.11447196e+00
  1.62353604e-01 -8.17103070e-01 -6.18470410e-01  1.08539315e+00
 -9.12837795e-01  3.32591955e-01 -9.37398183e-01  3.80382688e-0

eval_count: 15
var_params: [ 6.85676591e-01 -9.12514107e-01  1.08825401e+00  1.73740900e+00
 -1.14979269e+00 -5.62775003e-01  2.28232149e-01  3.83038064e-01
  1.25270709e+00  5.97235423e-01  2.30209944e-01 -1.29585080e-01
  3.63233578e-01  2.26276696e+00  2.84243368e-01 -6.21057867e-01
 -2.04856182e+00  1.03594956e+00  1.74330162e+00 -1.13141354e+00
 -1.02552510e-02 -1.08535631e+00  1.68004572e+00 -5.76162205e-01
 -1.78430883e+00 -2.21026995e-01  1.46152685e-01  1.07478206e+00
 -4.59351845e-02 -4.78414666e-02 -1.52263642e+00  4.84891687e-01
  2.40517337e+00 -3.35079932e-02  1.09506099e+00 -4.69896107e-01
 -4.38737636e-01  1.26083289e-01  7.33813010e-01  1.51476739e+00
  3.70235102e-01  5.98917531e-01 -3.46029133e-01 -1.00987786e+00
 -5.83920218e-01  1.00375635e+00 -2.76504727e-01  3.33913200e-01
 -6.88313164e-01 -4.59709270e-01 -1.38847882e+00 -2.11447196e+00
  1.62353604e-01 -8.17103070e-01 -6.18470410e-01  1.08539315e+00
 -9.12837795e-01  3.32591955e-01 -9.37398183e-01  3.80382688e-0

eval_count: 20
var_params: [ 6.85676591e-01 -9.12514107e-01  1.08825401e+00  1.73740900e+00
 -1.14979269e+00 -5.62775003e-01  2.28232149e-01  3.83038064e-01
  1.25270709e+00  5.97235423e-01  2.30209944e-01 -1.29585080e-01
  3.63233578e-01  2.26276696e+00  2.84243368e-01 -6.21057867e-01
 -2.04856182e+00  1.03594956e+00  1.74330162e+00 -1.31413541e-01
 -1.02552510e-02 -1.08535631e+00  1.68004572e+00 -5.76162205e-01
 -1.78430883e+00 -2.21026995e-01  1.46152685e-01  1.07478206e+00
 -4.59351845e-02 -4.78414666e-02 -1.52263642e+00  4.84891687e-01
  2.40517337e+00 -3.35079932e-02  1.09506099e+00 -4.69896107e-01
 -4.38737636e-01  1.26083289e-01  7.33813010e-01  1.51476739e+00
  3.70235102e-01  5.98917531e-01 -3.46029133e-01 -1.00987786e+00
 -5.83920218e-01  1.00375635e+00 -2.76504727e-01  3.33913200e-01
 -6.88313164e-01 -4.59709270e-01 -1.38847882e+00 -2.11447196e+00
  1.62353604e-01 -8.17103070e-01 -6.18470410e-01  1.08539315e+00
 -9.12837795e-01  3.32591955e-01 -9.37398183e-01  3.80382688e-0

...

eval_count: 105
var_params: [ 6.85676591e-01 -9.12514107e-01  1.08825401e+00  1.73740900e+00
 -1.14979269e+00 -5.62775003e-01  2.28232149e-01  3.83038064e-01
  1.25270709e+00  5.97235423e-01  2.30209944e-01 -1.29585080e-01
  3.63233578e-01  2.26276696e+00  2.84243368e-01 -6.21057867e-01
 -2.04856182e+00  1.03594956e+00  1.74330162e+00 -1.13141354e+00
 -1.02552510e-02 -1.08535631e+00  1.68004572e+00 -5.76162205e-01
 -1.78430883e+00 -2.21026995e-01  1.46152685e-01  2.07478206e+00
 -4.59351845e-02 -4.78414666e-02 -5.22636416e-01  4.84891687e-01
  2.40517337e+00  9.66492007e-01  1.09506099e+00 -4.69896107e-01
 -4.38737636e-01  1.26083289e-01  7.33813010e-01  1.51476739e+00
  3.70235102e-01  5.98917531e-01 -3.46029133e-01 -1.00987786e+00
 -5.83920218e-01  1.00375635e+00 -2.76504727e-01  3.33913200e-01
  3.11686836e-01 -4.59709270e-01 -1.38847882e+00 -2.11447196e+00
  1.62353604e-01 -8.17103070e-01 -6.18470410e-01  1.08539315e+00
 -9.12837795e-01  3.32591955e-01 -9.37398183e-01  3.80382688e-

eval_count: 110
var_params: [ 1.86382112e-01 -9.12514107e-01  1.08825401e+00  1.73740900e+00
 -1.14979269e+00 -5.62775003e-01  2.28232149e-01  3.83038064e-01
  1.25270709e+00  5.97235423e-01  2.30209944e-01 -1.29585080e-01
  3.63233578e-01  2.26276696e+00  2.84243368e-01 -6.21057867e-01
 -2.04856182e+00  1.03594956e+00  1.74330162e+00 -1.13141354e+00
 -1.02552510e-02 -1.08535631e+00  1.68004572e+00 -5.76162205e-01
 -1.78430883e+00 -2.21026995e-01  1.46152685e-01  2.07478206e+00
 -4.59351845e-02 -4.78414666e-02 -5.22636416e-01  4.84891687e-01
  2.43172564e+00  9.66492007e-01  1.09506099e+00 -4.69896107e-01
 -4.38737636e-01  1.26083289e-01  7.33813010e-01  1.51476739e+00
  3.70235102e-01  5.98917531e-01 -3.46029133e-01 -1.00987786e+00
 -5.83920218e-01  1.00375635e+00 -2.76504727e-01  3.33913200e-01
  3.11686836e-01 -4.59709270e-01 -1.38847882e+00 -2.11447196e+00
  1.62353604e-01 -8.17103070e-01 -6.18470410e-01  1.08539315e+00
 -9.12837795e-01  3.32591955e-01 -9.37398183e-01  3.80382688e-

eval_count: 115
var_params: [ 0.69982348 -0.91013672  1.22820688  1.737409   -1.14979269 -0.562775
  0.22823215  0.38303806  1.25270709  0.59720085  0.22963655 -0.13002729
  0.36473834  2.26151235  0.28494231 -0.62113997 -2.0491817   2.02443569
  1.74102831 -1.13260529 -0.01595246 -1.08573505  1.67608862 -0.57863983
 -1.78836091 -0.23075287  0.1432524   2.0760249  -0.04990519 -0.04868642
 -0.5192162   0.48279125  2.39861338  0.96653419  1.09174958 -0.4705037
 -0.4436541   0.12096371  0.72127613  1.5119949   0.368822    0.59642746
 -0.35227008 -1.01118948 -0.58546385  1.00277438 -0.28816438  0.33136487
  0.31187853 -0.46385079 -1.38892131 -2.119224    0.16012918 -0.81833431
 -0.61935298  1.08084963 -0.91351168  0.32862695 -0.94224339  0.03293024
  0.39347376  0.36192964 -1.47945547  0.80063322 -0.29875241  0.90817696
  1.12825705  0.01657297 -0.75865323 -1.85013951 -2.13242749  0.57112413
 -0.44256153 -1.603834   -1.22189616 -1.24074882  1.24558766 -0.59827141
  0.0675234   0.78202131 -

eval_count: 121
var_params: [ 0.68714478 -0.91226738  1.08853764  1.737409   -1.14979269 -0.562775
  0.22823215  0.38303806  1.25270709  0.59723184  0.2295916   0.87039666
  0.36338974  2.26261416  0.28431591 -0.62106639 -2.04862615  1.03530282
  1.74306569 -1.13153722 -0.01084652 -1.08539561  1.67963504 -0.57641934
 -1.78472937 -0.22203636  0.14585169  2.07491104 -0.0463472  -0.04792916
 -0.52228146  0.4846737   2.40449257  0.96649638  1.09471732 -0.46995916
 -0.43924787  0.12555197  0.73251191  1.51447966  0.37008845  0.59865911
 -0.34667683 -1.01001398 -0.58408042  1.00365444 -0.27771478  0.33364873
  0.31170673 -0.46013908 -1.38852474 -2.11496513  0.16212275 -0.81723085
 -0.618562    1.08492161 -0.91290773  0.33218046 -0.93790103  0.03750815
  0.39456556  0.36811421 -1.47845651  0.80042929 -0.29570171  0.92341767
  1.13063714  0.01774372 -0.75713195 -1.84895867 -2.12897526  0.57375603
 -0.44283479 -1.6016829  -1.21176195 -1.24013108  1.24703339 -0.59657055
  0.08012767  0.78181082 

eval_count: 127
var_params: [ 0.72007191 -0.90673397  1.09489859  1.737409   -1.14979269 -0.562775
  0.22848972  0.38329564  1.32966922  0.60145127  0.21572407  0.87145377
  0.29278622  2.25918729  0.2859427  -0.62125748 -2.05006894  1.02079817
  1.73777454 -1.13431105 -0.02410687 -1.08627713  1.67042483 -0.58218605
 -1.79416064  0.76347442  0.13910122  2.07780378 -0.05558746 -0.04989581
 -0.51432085  0.4797849   2.38922407  0.96659457  1.08700997 -0.47137334
 -0.45069102  0.11363607  0.70333211  1.50802665  0.36679943  0.59286342
 -0.36120275 -1.01306679 -0.58767326  1.00136888 -0.30485283  0.32771744
  0.31215291 -0.46977855 -1.38955466 -2.12602557  0.15694537 -0.82009658
 -0.6206162   1.07434649 -0.9144762   0.32295184 -0.94917832  0.02561912
  0.39173011  0.35205262 -1.48105087  0.80095891 -0.30362451  0.8838369
  1.12445595  0.01470322 -0.76108278 -1.85202536 -2.13794085  0.56692087
 -0.44212514 -1.6072694  -1.23808093 -1.24173537  1.24327877 -0.60098775
  0.04739385  0.78235746 -

In [ ]:
print(result)

## Storing the optimal parameters for grading

Once the training step of the vqc algorithm is done we obtain the optimal parameters for our specific variational form. For the grading function to be able to access these optimal parameters you will need to follow the steps below. 

* **Step 1**: Run the cell below with `print(repr(vqc.optimal_params))`. 
* **Step 2**: Copy the matrix of optimal parameters and store it in the variable `optimal_parameters` inside the function `return_optimal_params()` in the next cell. This will enable us to extract it while calculating the accuracy your the model during grading. Given below is a pictographical explanation of the same:  

<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/opt_params456b075.png" width="800">

In [217]:
print(repr(vqc.optimal_params))

array([ 0.54632732,  0.40538044, -1.01295329,  1.49487678,  1.64501921,
        0.59286665,  2.12784723,  1.21054489, -0.5643853 ,  0.95213542,
       -0.27556139, -1.83490042, -0.01200562,  3.75627203,  1.43557252,
       -0.39997671, -0.5545942 ,  0.46746988, -1.27218015,  0.0318519 ,
        0.87543005, -0.25437162, -1.34021757, -2.08737164,  2.04253738,
        0.83175021,  0.77956833, -0.17427723,  2.16326879, -1.5828532 ,
        1.17114289, -1.17920076, -1.45410869, -0.89369175,  0.01718402,
       -1.59692096,  0.34737521, -2.48543596,  0.74288619,  0.28864511,
        0.57559697,  0.51204554, -0.92355601,  0.49001691, -0.6171768 ,
        0.14289195,  1.87852521,  0.44203041,  2.02773329,  1.01900935,
        1.40938457, -0.3939512 ,  0.50795543,  0.43727053,  0.64402482,
        0.43716062, -2.43465227,  0.81417097,  0.38247537, -0.64960273,
       -0.71656997,  0.64017647,  0.91821865, -0.04276097, -2.2802225 ,
       -0.90311979,  1.9641994 , -0.9093311 , -0.08099036,  0.13

In [1]:
%%write_and_run optimal_params.py
# # the write_and_run function writes the content in this cell into the file "optimal_params.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
import numpy as np
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def return_optimal_params():
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    
    optimal_parameters = [0.68714478,-0.91226738,1.08853764,1.737409,-1.14979269,-0.562775
,0.22823215,0.38303806,1.25270709,0.59723184,0.2295916,0.87039666
,0.36338974,2.26261416,0.28431591,-0.62106639,-2.04862615,1.03530282
,1.97521809,-1.57437496,-0.01084652,-1.08539561,1.32191418,-0.57641934
,-1.43539316,-0.22203636,0.14585169,2.07491104,-0.0463472,-0.04792916
,-0.52228146,0.4846737,2.40449257,0.96649638,1.09471732,-0.46995916
,-0.43924787,0.12555197,0.73251191,1.51447966,0.37008845,0.59865911
,-0.34667683,-1.01001398,-0.58408042,1.00365444,-0.27771478,0.33364873
,0.31170673,-0.46013908,-1.38852474,-2.11496513,0.16212275,-0.81723085
,-0.618562,1.08492161,-0.91290773,0.33218046,-0.93790103,0.03750815
,0.39456556,0.36811421,-1.47845651,0.80042929,-0.29570171,0.92341767
,1.13063714,0.01774372,-0.75713195,-1.84895867,-2.12897526,0.57375603
,-0.44283479,-1.6016829,-1.21176195,-1.24013108,1.24703339,-0.59657055
,0.08012767,0.78181082,-0.7172605,0.91874437,-0.35211607,-0.99370252
,-0.99676725,0.51809799,-0.30044022,-2.74560962,-1.21566705,0.66392358
,0.08044294,-0.84033885,-0.17474038,-1.74582009,-0.15756778,1.09758557
,-0.70618137,0.95488175,0.65264357,-1.58953711,-0.26958789,3.72474648
,0.2332852,0.29723226,-0.54259352,-0.00400235,-0.15284817,0.9574589]
    
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    return np.array(optimal_parameters)

UsageError: Cell magic `%%write_and_run` not found.


## Submission

Before we go any further, check that you have the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` in the **same working directory as this notebook**. If you do not, then go back to the start and run the notebook making sure you have filled in the code where its required. When you run the cell below, all the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` are combined into one file named **"answer.py"**. Now your working directory will have four python (.py) files out of which **"answer.py"** is the submission file: 
* `answer.py` <- upload this file onto HackerEarth and click on "Submit and Evaluate"
* `feature_map.py`
* `variational_circuit.py`
* `optimal_params.py`

In [219]:
solution = ['feature_map.py','variational_circuit.py','optimal_params.py']
file = open("answer.py","w")
file.truncate(0)
for i in solution:    
    with open(i) as f:
        with open("answer.py", "a") as f1:
            for line in f:
                f1.write(line)
file.close()

## Grading Function

Given below is the grading function that we shall use to grade your submission with a test dataset that is of the same format as `challenge_dataset_4_9.csv`. You can use it to grade your submission by extracting a few points out of the `challenge_dataset_4_9.csv` to get a basic idea of how your model is performing. 

In [220]:
#imports required for the grading function 
from qiskit import *
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit.aqua.components.variational_forms import VariationalForm
import numpy as np

### Working of the grading function

The grading function `grade()` takes as **input**: 

* `test_data`: (`np.ndarray`) -- **no. of datapoints $\times$ dimension of data** : the datapoints against which we want to test our model. 


* `test_labels`: (`np.ndarray`) -- **no. of datapoints $\times$ 1** : A column vector with each entry either 0 or 1 as entries.


* `feature_map`: (`QuantumCircuit` or `FeatureMap`) -- A quantum feature map which is the output of `feature_map()` defined earlier.


* `variational_form`: (`QuantumCircuit` or `VariationalForm`) -- A variational form which is the output of `variational_circuit()` defined earlier.


* `optimal_params`: (`numpy.ndarray`) -- the optimal parameters obtained after running the VQC algorithm above. These are the values obtained when the function `return_optimal_params()` is run. 


* `find_circuit_cost` : (`bool`) -- Calculates the circuit cost if set to `True`. Circuit cost is calculated by converting the circuit to the basis gate set `\[ 'u3', 'cx'\]` and then applying the formula **cost = 1$\times$(no.of u3 gates) + 10$\times$(no.of cx gates)**.


* `verbose` : (`bool`) -- prints the result message if set to `True`.

And gives as **output**: 

* `model_accuracy` : (`numpy.float64`) -- percent accuracy of the model. 


* `circuit_cost`: (`int`) -- circuit cost as explained above.


* `ans`: (`tuple`) -- Output of the `VQC.predict()` method. 


* `result_msg`: (`str`) -- Result message which also outputs the error message in case of one.


* `unrolled_circuit`: (`QuantumCircuit` or `None`) -- the circuit obtained after unrolling the full VQC circuit and substituting the optimal parameters to the basis gate set `\[ 'u3', 'cx'\]`.

**Note:** if you look inside the `grade()` function in Section 2 you'll see that we have initialized a COBYLA optimizer though the prediction step will not require one. Similarily we have given a dataset to `training dataset`. Both of these are dummy variables. The reason for this is because these are not optional variables the `VQC` class instantiation.  

In [221]:
def grade(test_data, test_labels, feature_map, variational_form, optimal_params, find_circuit_cost=True, verbose=True):
    seed = 10598
    model_accuracy = None 
    circuit_cost=None 
    ans = None
    unrolled_circuit = None
    result_msg=''
    data_dim = np.array(test_data).shape[1]
    dataset_size = np.array(test_data).shape[0]
    dummy_training_dataset=training_input = {'A':np.ones((2,data_dim)), 'B':np.ones((2, data_dim))}
    
    # converting 4's to 0's and 9's to 1's for checking 
    test_labels_transformed = np.where(test_labels==4, 0., 1.)
    max_qubit_count = 6
    max_circuit_cost = 2000
    
    # Section 1
    if feature_map is None:
        result_msg += 'feature_map variable is None. Please submit a valid entry' if verbose else ''
    elif variational_form is None: 
        result_msg += 'variational_form variable is None. Please submit a valid entry' if verbose else ''
    elif optimal_params is None: 
        result_msg += 'optimal_params variable is None. Please submit a valid entry' if verbose else ''
    elif test_data is None: 
        result_msg += 'test_data variable is None. Please submit a valid entry' if verbose else ''
    elif test_labels is None: 
        result_msg += 'test_labels variable is None. Please submit a valid entry' if verbose else ''
    elif not isinstance(feature_map, (QuantumCircuit, FeatureMap)):
        result_msg += 'feature_map variable should be a QuantumCircuit or a FeatureMap not (%s)' % \
                      type(feature_map) if verbose else ''
    elif not isinstance(variational_form, (QuantumCircuit, VariationalForm)):
        result_msg += 'variational_form variable should be a QuantumCircuit or a VariationalForm not (%s)' % \
                      type(variational_form) if verbose else ''
    elif not isinstance(test_data, np.ndarray):
        result_msg += 'test_data variable should be a numpy.ndarray not (%s)' % \
                      type(test_data) if verbose else ''
    elif not isinstance(test_labels, np.ndarray):
        result_msg += 'test_labels variable should be a numpy.ndarray not (%s)' % \
                      type(test_labels) if verbose else ''
    elif not isinstance(optimal_params, np.ndarray):
        result_msg += 'optimal_params variable should be a numpy.ndarray not (%s)' % \
                      type(optimal_params) if verbose else ''
    elif not dataset_size == test_labels_transformed.shape[0]:
        result_msg += 'Dataset size and label array size must be equal'
    # Section 2
    else:
        
        # setting up COBYLA optimizer as a dummy optimizer
        from qiskit.aqua.components.optimizers import COBYLA
        dummy_optimizer = COBYLA()

        # setting up the backend and creating a quantum instance
        backend = Aer.get_backend('qasm_simulator')
        backend_options = {"method": "statevector"}
        quantum_instance = QuantumInstance(backend, 
                                           shots=2000, 
                                           seed_simulator=seed, 
                                           seed_transpiler=seed, 
                                           backend_options=backend_options)

        # creating a VQC instance and running the VQC.predict method to get the accuracy of the model 
        vqc = VQC(optimizer=dummy_optimizer, 
                  feature_map=feature_map, 
                  var_form=variational_form, 
                  training_dataset=dummy_training_dataset)
        
        from qiskit.transpiler import PassManager
        from qiskit.transpiler.passes import Unroller
        pass_ = Unroller(['u3', 'cx'])
        pm = PassManager(pass_)
        # construct circuit with first datapoint
        circuit = vqc.construct_circuit(data[0], optimal_params)
        unrolled_circuit = pm.run(circuit)
        gates = unrolled_circuit.count_ops()
        if 'u3' in gates: 
            circuit_cost = gates['u3']
        if 'cx' in gates: 
            circuit_cost+= 10*gates['cx']
        
        if circuit.num_qubits > max_qubit_count:
            result_msg += 'Your quantum circuit is using more than 6 qubits. Reduce the number of qubits used and try again.'
        elif circuit_cost > max_circuit_cost:
            result_msg += 'The cost of your circuit is exceeding the maximum accpetable cost of 2000. Reduce the circuit cost and try again.'
        else: 
            
            ans = vqc.predict(test_data, quantum_instance=quantum_instance, params=np.array(optimal_params))
            model_accuracy = np.sum(np.equal(test_labels_transformed, ans[1]))/len(ans[1])

            result_msg += 'Accuracy of the model is {}'.format(model_accuracy) if verbose else ''
            result_msg += ' and circuit cost is {}'.format(circuit_cost) if verbose else ''
            
    return model_accuracy, circuit_cost, ans, result_msg, unrolled_circuit

## Process of grading using a dummy grading dataset

Let us create a dummy grading dataset with features and labels `grading_features` and `grading_labels` created from the last 2000 datapoints from `data_features` and `data_labels`so that we can a rough estimate of our accuaracy. It must be noted that this may not be a balanced dataset, i.e, may not have equal number of `4`'s and `9`'s and is not best practice. This is only given for the purpose of the demo of `grade()` function. In the final scoring done on HackerEarth, the testing dataset used will have a balanced number of class labels `4` and `9`.

In [222]:
grading_dataset_size=2000    # this value is not per digit but in total
grading_features = data_features[-grading_dataset_size:]
grading_labels = data_labels[-grading_dataset_size:]

In [223]:
start = time.process_time()

accuracy, circuit_cost, ans, result_msg, full_circuit  =  grade(test_data=grading_features, 
                                                                test_labels=grading_labels, 
                                                                feature_map=feature_map(), 
                                                                variational_form=variational_circuit(), 
                                                                optimal_params=return_optimal_params())

print("time taken: {} seconds".format(time.process_time() - start))
print(result_msg)

time taken: 169.226498146003 seconds
Accuracy of the model is 0.81 and circuit cost is 876


You can also check your **accuracy**, **circuit_cost** and **full_circuit** which is the result of combining the feature map and variational circuit and unrolling into the basis \['u3', 'cx'\].

In [52]:
print("Accuracy of the model: {}".format(accuracy))
print("Circuit Cost: {}".format(circuit_cost))
print("The complete unrolled circuit: ")
full_circuit.draw()

Accuracy of the model: 0.6835
Circuit Cost: 648
The complete unrolled circuit: 


┌───────────────┐                                                 »
q_0: ┤ U3(pi/2,0,pi) ├──────────────────────■───────────────────────■──»
     ├───────────────┤┌──────────────────┐┌─┴─┐┌─────────────────┐┌─┴─┐»
q_1: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,-0.99523) ├┤ X ├┤ U3(0,0,0.99523) ├┤ X ├»
     ├───────────────┤└┬───────────────┬─┘└───┘└─────────────────┘└───┘»
q_2: ┤ U3(pi/2,0,pi) ├─┤ U3(0,0,2.597) ├───────────────────────────────»
     └───────────────┘ └───────────────┘                               »
c_0: ══════════════════════════════════════════════════════════════════»
                                                                       »
c_1: ══════════════════════════════════════════════════════════════════»
                                                                       »
c_2: ══════════════════════════════════════════════════════════════════»
                                                                       »
«     ┌─────────────────┐                                                ┌───┐»
«q_0: ┤ U3(0,0,-0.9614) ├────────────────────────────────────────────────┤ X ├»
«     └─────────────────┘                       ┌───────────────────────┐└─┬─┘»
«q_1: ─────────■─────────────────────────────■──┤ U3(0.8228,-pi/2,pi/2) ├──┼──»
«            ┌─┴─┐       ┌────────────────┐┌─┴─┐└───────────────────────┘  │  »
«q_2: ───────┤ X ├───────┤ U3(0,0,-2.597) ├┤ X ├───────────────────────────■──»
«            └───┘       └────────────────┘└───┘                              »
«c_0: ════════════════════════════════════════════════════════════════════════»
«                                                                             »
«c_1: ════════════════════════════════════════════════════════════════════════»
«                                                                             »
«c_2: ════════════════════════════════════════════════════════════════════════»
«                                                                             »
«     ┌────────────────┐                    ┌───┐   ┌──────────────────┐   »
«q_0: ┤ U3(0,0,0.9614) ├────────────────────┤ X ├───┤ U3(4,-pi/2,pi/2) ├───»
«     ├───────────────┬┘┌──────────────────┐└─┬─┘   └──────────────────┘   »
«q_1: ┤ U3(pi/2,0,pi) ├─┤ U3(0,0,-0.99523) ├──┼────────────────────────────»
«     └───────────────┘ └──────────────────┘  │  ┌────────────────────────┐»
«q_2: ────────────────────────────────────────■──┤ U3(0.90163,-pi/2,pi/2) ├»
«                                                └────────────────────────┘»
«c_0: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«c_1: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«c_2: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«     ┌───────────────┐                                         »
«q_0: ┤ U3(pi/2,0,pi) ├────────■─────────────────────────────■──»
«     └───────────────┘      ┌─┴─┐      ┌─────────────────┐┌─┴─┐»
«q_1: ───────────────────────┤ X ├──────┤ U3(0,0,0.99523) ├┤ X ├»
«     ┌───────────────┐┌─────┴───┴─────┐└─────────────────┘└───┘»
«q_2: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,2.597) ├────────────────────────»
«     └───────────────┘└───────────────┘                        »
«c_0: ══════════════════════════════════════════════════════════»
«                                                               »
«c_1: ══════════════════════════════════════════════════════════»
«                                                               »
«c_2: ══════════════════════════════════════════════════════════»
«                                                               »
«     ┌─────────────────┐                                                ┌───┐»
«q_0: ┤ U3(0,0,-0.9614) ├────────────────────────────────────────────────┤ X 